# builder

An `ipywidgets`-based builder for `jupyak` config files.

In [ ]:
if "pyodide" in __import__("sys").modules:
    %pip install jupyak ipywidgets tomli-w

In [ ]:
import urllib.parse

import importnb
import traitlets as T
from IPython import display as D

with importnb.Notebook():
    import jupyak.tasks._yak as Y
    from jupyak.tasks import load_tasks

In [ ]:
try:
    import ipywidgets as W
except ImportError:
    pass

In [ ]:
URL = "https://github.com/deathbeds/jupyak/new/main"

In [ ]:
as_column = dict(layout=dict(flex_flow="column", align_items="stretch"))

In [ ]:
def make_select(owner: "W.Widget", trait_name: str, **kwargs):
    select = W.Select(**kwargs)
    T.link((owner, trait_name), (select, "value"))
    return select

In [ ]:
def make_split_textarea(owner: "W.Widget", trait_name: str, **kwargs):
    text = W.Textarea(**kwargs)
    error = W.HTML(layout=dict(display="none"))

    def on_value(*_):
        new_value = "\n".join(getattr(owner, trait_name))
        if new_value != text.value:
            text.value = new_value

    def on_text(*_):
        new_value = tuple(text.value.split("\n"))
        if getattr(owner, trait_name) != new_value:
            try:
                setattr(owner, trait_name, new_value)
                error.layout.display = "none"
                error.value = ""
            except Exception as err:
                error.value = f"<details><summary>errors...</summary>{err}</details>"
                error.layout.display = "block"

    owner.observe(on_value, trait_name)
    text.observe(on_text, "value")
    on_value()
    return W.VBox([text, error])

In [ ]:
def make_github(gh: Y.GitHub):
    label = W.HTML("<h4>github</h4>")
    url = W.Text(description="URL", **as_column)
    baseline = W.Text(description="baseline", **as_column)
    merge_with = make_split_textarea(
        gh,
        "merge_with",
        description="merge with",
        placeholder="\n".join(
            ["pull/{:number}", "tree/{:branch}", "releases/tags/{:tag}"]
        ),
        rows=3,
        **as_column,
    )
    strategy = make_select(
        gh,
        "merge_strategy",
        description="strategy",
        options=["", "ort", "resolve", "octopus", "ours", "subtree"],
        rows=1,
        **as_column,
    )
    options = make_split_textarea(
        gh,
        "merge_options",
        description="options",
        placeholder="\n".join(
            ["theirs", "ours", "diff-algorithm=[patience|minimal|histogram|myers]"]
        ),
        rows=3,
        **as_column,
    )
    T.link((gh, "baseline"), (baseline, "value"))
    T.link((gh, "url"), (url, "value"))
    return W.HBox([label, url, baseline, merge_with, strategy, options])

In [ ]:
def make_repo(repo: Y.Repo, yak: Y.Yak):
    label = W.HTML()
    name = W.Text(description="repo name")
    remove = W.Button(icon="trash", button_style="warning")
    gh = make_github(repo.github)

    T.link((repo, "name"), (name, "value"))
    T.dlink((repo, "name"), (label, "value"), lambda x: f"<h3>{x}</h3>")
    T.dlink((repo, "name"), (remove, "description"), lambda x: f"remove {x}")

    @remove.on_click
    def on_remove(*_):
        yak.repos = {k: v for k, v in yak.repos.items() if v != repo}

    return W.VBox(
        [
            W.HBox([label, W.HBox([name, remove])]),
            gh,
        ]
    )

In [ ]:
def make_add_known(yak: Y.Yak):
    old_repos = yak.repos
    select_repo = W.Select(options=old_repos, rows=1)
    add_known = W.Button(
        icon="wrench", description="customize repo", button_style="success"
    )

    def on_repos_change(*_):
        select_repo.options = {
            k: v for k, v in old_repos.items() if v not in yak.repos.values()
        }

    yak.observe(on_repos_change, "repos")

    @add_known.on_click
    def _on_add_known(*_):
        repo = select_repo.value
        if repo is None:
            return
        yak.repos = {**yak.repos, repo.name: repo}
        select_repo.options = {
            k: v for k, v in dict(select_repo.options).items() if v != repo
        }

    return W.HBox([select_repo, add_known])

In [ ]:
def make_add_custom(yak: Y.Yak):
    custom_name = W.Text(placeholder="repo name")
    add_custom = W.Button(
        icon="plus-square", description="add custom repo", button_style="primary"
    )
    return W.HBox([custom_name, add_custom])

In [ ]:
def make_repos(yak: Y.Yak):
    add_known = make_add_known(yak)
    add_custom = make_add_custom(yak)
    repos = W.VBox()
    label = W.HTML()

    def on_repo(*_):
        children = {r.repo: r for r in repos.children if r in yak.repos.values()}
        [
            children.update({repo: make_repo(repo, yak)})
            for repo in yak.repos.values()
            if repo not in children
        ]
        repos.children = tuple(children.values())
        label.value = f"<h2>repos ({len(repos.children)})</h2>"

    yak.observe(on_repo, "repos")
    on_repo()
    return W.VBox([W.HBox([label, add_known, add_custom]), repos])

In [ ]:
def make_serialized(yak: Y.Yak):
    observing = {yak: True, yak.lite: True, yak.env: True}
    link = W.HTML(style={"color": "var(--jp-brand-color1)"})
    toml = W.Textarea(
        description=".toml",
        rows=20,
        layout=dict(
            flex_flow="column", align_items="stretch", width="99%", display="none"
        ),
    )

    def on_change(*_):
        import tomli_w

        toml_text = tomli_w.dumps(yak.to_dict())
        if not toml_text:
            toml.layout.display = link.layout.display = "none"
            return
        toml.value = toml_text
        params = {"filename": "jupyak_config.toml", "value": toml_text}
        url = f"{URL}?{urllib.parse.urlencode(params)}"
        link.value = f"""
        <a href="{url}" target="_blank" class="jpyk-big-button">
            <i class="fab fa-github-alt"></i>
            start pull request
        </a>
        """
        toml.layout.display = link.layout.display = "block"
        for repo in yak.repos.values():
            for src in [repo, repo.github, repo.py, repo.js, repo.lite]:
                if src and src not in observing:
                    observing[src] = True
                    src.observe(on_change)

    [src.observe(on_change) for src in observing]
    return W.VBox(
        [
            W.HTML("<h2>jupyak_config</h2>"),
            toml,
            link,
        ]
    )

In [ ]:
def make_style():
    return W.HTML(
        """
        <style>
        .jpyk-big-button {
            font-size: var(--jp-content-font-size5);
            text-decoration: underline;
            color: var(--jp-brand-color1);
        }
        </style>
        """
    )

In [ ]:
def make_builder():
    yak = Y.Yak({"repos": {}})
    old_repos = yak.repos
    repos = make_repos(yak)
    yak.repos = {}
    style = make_style()
    serialized = make_serialized(yak)
    box = W.VBox([style, repos, serialized])
    return box

In [ ]:
if __name__ == "__main__":
    load_tasks()
    D.display(make_builder())